In [ ]:
import random
import numpy as np


CLASSES = {"Toán": ["GV_Toan1", "GV_Toan2"], "Tin học": ["GV_Tin1", "GV_Tin2"]}
REQUIRED_HOURS_PER_TEACHER = 5 # Mỗi GV dạy 5 tiết/tuần
# Giả sử 6 ngày (Thứ 2 - Thứ 7), mỗi ngày 5 tiết = 6*5= 30 Slots
SLOTS = [f"Day{d}_T{t}" for d in range(2, 8) for t in range(1, 6)]
POPULATION_SIZE = 50
GENERATIONS = 100

# Tạo danh sách Giáo viên, Môn học cần sắp xếp
REQUIRED_LESSONS = []
for subject, teachers in CLASSES.items():
    for teacher in teachers:
        REQUIRED_LESSONS.extend([(teacher, subject)] * REQUIRED_HOURS_PER_TEACHER)
NUM_LESSONS = len(REQUIRED_LESSONS)

# ---  HÀM TÍNH ĐỘ THÍCH NGHI (FITNESS) ---
def calculate_fitness(chromosome):
    """Tính điểm thích nghi dựa trên ràng buộc cứng."""
    score = 1000 # Điểm khởi tạo

    # Ràng buộc CỨNG: Không có hai tiết học nào dùng chung một Slot.
    slot_usage = {}
    for slot, teacher, subject in chromosome:
        if slot in slot_usage:
            score -= 500 # Phạt nặng nếu trùng Slot
        slot_usage[slot] = teacher

    # Ràng buộc CỨNG: Số tiết/GV phải đúng yêu cầu.
    teacher_hour_count = {t: 0 for sub_list in CLASSES.values() for t in sub_list}
    for slot, teacher, subject in chromosome:
        teacher_hour_count[teacher] += 1

    for count in teacher_hour_count.values():
        score -= abs(count - REQUIRED_HOURS_PER_TEACHER) * 100 # Phạt số tiết thiếu/thừa

    # Ràng buộc MỀM: Không dạy quá 2 tiết liên tiếp.

    return max(1, score)

# ---  HÀM KHỞI TẠO QUẦN THỂ ---
def initialize_population(pop_size, lessons, slots):
    population = []
    for _ in range(pop_size):
        # Chọn ngẫu nhiên N_LESSONS slots từ 30 slots có sẵn
        schedule_slots = random.sample(slots, NUM_LESSONS)

        # Gán ngẫu nhiên các đơn vị dạy vào các slots đã chọn
        random.shuffle(lessons)

        chromosome = []
        for i in range(NUM_LESSONS):
            teacher, subject = lessons[i]
            slot = schedule_slots[i]

            chromosome.append((slot, teacher, subject))

        population.append(chromosome)
    return population

# --- TOÁN TỬ DI TRUYỀN ---
def select_parents(population, fitness_scores):
    # Tournament Selection: Chọn ngẫu nhiên 3 cá thể và chọn cá thể tốt nhất trong 3
    candidates = random.sample(list(zip(population, fitness_scores)), 3)
    candidates.sort(key=lambda x: x[1], reverse=True)
    return candidates[0][0]

def crossover(parent1, parent2):   #  Hoán đổi một đoạn của lịch trình
    point = NUM_LESSONS // 2
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def mutate(chromosome):        # Hoán đổi Slot của hai Gen ngẫu nhiên (Giữ nguyên Giáo viên và Môn học)
    if random.random() < 0.05: # Tỷ lệ đột biến 5%
        idx1, idx2 = random.sample(range(NUM_LESSONS), 2)

        # Hoán đổi Slot:
        slot1 = chromosome[idx1][0]
        slot2 = chromosome[idx2][0]

        # Tạo Gen mới (Slot được hoán đổi)
        chromosome[idx1] = (slot2, chromosome[idx1][1], chromosome[idx1][2])
        chromosome[idx2] = (slot1, chromosome[idx2][1], chromosome[idx2][2])
    return chromosome

# --- LÕI THUẬT TOÁN DI TRUYỀN ---
def genetic_algorithm_scheduling(pop_size, generations):
    population = initialize_population(pop_size, REQUIRED_LESSONS, SLOTS)

    for generation in range(generations):
        fitness_scores = [calculate_fitness(c) for c in population]
        best_index = np.argmax(fitness_scores)
        best_chromosome = population[best_index]
        best_fitness = fitness_scores[best_index]

        # Nếu tìm được lịch trình tốt thì dừng lại
        if best_fitness >= 900:
            print(f"Lời giải tối ưu tìm thấy ở thế hệ {generation}!")
            return best_chromosome

        new_population = [best_chromosome] # Giữ lại cá thể tốt nhất

        while len(new_population) < pop_size:
            parent1 = select_parents(population, fitness_scores)
            parent2 = select_parents(population, fitness_scores)

            child1, child2 = crossover(parent1, parent2)

            new_population.append(mutate(child1))
            if len(new_population) < pop_size:
                 new_population.append(mutate(child2))

        population = new_population

    return best_chromosome


if __name__ == "__main__":
    print("---GIẢI THUẬT DI TRUYỀN CHO SẮP LỊCH ---")
    final_schedule = genetic_algorithm_scheduling(POPULATION_SIZE, GENERATIONS)

    if final_schedule:
        print("\nLỊCH TRÌNH TỐT NHẤT:")
        schedule_dict = {}
        for slot, teacher, subject in final_schedule:
            schedule_dict[slot] = f"{subject} ({teacher})"

        # Sắp xếp theo thứ tự ngày và tiết
        sorted_schedule = sorted(schedule_dict.items(),
                                 key=lambda x: (int(x[0].split('_')[0].replace('Day', '')),
                                                int(x[0].split('_')[1].replace('T', ''))))

        for slot, info in sorted_schedule:
             print(f"{slot.ljust(8)}: {info}")

        print(f"\nĐiểm thích nghi cuối cùng: {calculate_fitness(final_schedule)}")